In [ ]:
# 1) Installing  dependencies for model training
!pip install ultralytics supervision roboflow

In [ ]:
#loading the dataset

from roboflow import Roboflow
rf = Roboflow(api_key="Cuz1HOeij6Ko2JrZf650")
project = rf.workspace("cvprac").project("qrcodereader-hydhr")
version = project.version(1)
dataset = version.download("yolov8")

In [ ]:
#training loop

from ultralytics import YOLO

# Load a pretrained YOLOv8 model (small version is faster, you can also try 'yolov8m' or 'yolov8l')
model = YOLO("yolov8s.pt")

# Train
model.train(
    data="/content/qrcodereader-1/data.yaml",  # path to dataset config
    epochs=10,          # number of training epochs (increase if needed)
    imgsz=640,          # input image size
    batch=16,           # batch size (adjust if GPU RAM is low)
    name="barcode_detector"
)


In [ ]:
#running validation
model.val()


In [ ]:
#testing

results = model.predict("/content/qrcodereader-1/test/images", save=True, conf=0.5)


In [ ]:
#script for loading the pre trained model weights

# Load the trained model
trained_model = YOLO("runs/detect/barcode_detector/weights/best.pt")


In [ ]:
from google.colab import files
uploaded = files.upload()  # choose your image


In [ ]:
results = model.predict(source=list(uploaded.keys())[0], save=True, conf=0.5)






In [ ]:
#croping the bounded boxed parts

import cv2

# Load original image
image = cv2.imread("/content/runs/detect/barcode_detector4/img_qr.jpg")

# Loop over results
for r in results:
    boxes = r.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2] format
    class_ids = r.boxes.cls.cpu().numpy().astype(int)

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)

        # Crop the detected region
        cropped = image[y1:y2, x1:x2]

        # Save each cropped image
        cv2.imwrite(f"crop_{i}_class{class_ids[i]}.jpg", cropped)


In [ ]:
!pip install pyzbar pillow
!apt-get install -y libzbar0



In [ ]:
from pyzbar.pyzbar import decode
from PIL import Image





In [ ]:
#scanning the content of qrcode/barcode

decoded_objects = decode(Image.fromarray(cropped))
if decoded_objects:
    for obj in decoded_objects:
        print(f"Detected {obj.type}: {obj.data.decode('utf-8')}")
else:
    print("No QR/Barcode detected.")